In [1]:
def changes_from_press(stock_data, press_date, period):
    next_day = timedelta(days=1)
    time_after_release = timedelta(days=period)
    

    next_trading_day = press_date
    day_1_price = stock_data[stock_data['Date'] == press_date]['Open'].values

    while not day_1_price:
        next_trading_day = next_trading_day + next_day

        day_1_price = stock_data[stock_data['Date'] == (next_trading_day)]['Open'].values

    next_trading_day = next_trading_day + time_after_release
    day_2_price = stock_data[stock_data['Date'] == (next_trading_day)]['Close'].values

    while not day_2_price:
        next_trading_day = next_trading_day + next_day

        day_2_price = stock_data[stock_data['Date'] == (next_trading_day + next_day)]['Close'].values


    pct_change = ((day_2_price - day_1_price) / day_2_price)*100
    pct_change = pct_change[0]
    return pct_change

def get_df(ticker):
    
    stock_data = yf.Ticker(ticker)

    # get historical market data
    stock_hist = stock_data.history(period="max")

    stock_hist.reset_index(inplace=True)

    stock_hist['Date'] = pd.to_datetime(stock_hist['Date']).dt.date

    stock_hist['Pct_Close'] = stock_hist['Close'].pct_change()*100

    return stock_hist

In [8]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import re
import json
from requests_html import HTMLSession
import nest_asyncio




data = []


ticker = 'ba'

stock_hist = get_df(ticker)

years = [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011]


for year in years:

    nest_asyncio.apply()

    s = HTMLSession()
    url=f'https://investors.boeing.com/investors/news/{year}/default.aspx'
    r = s.get(url)
    ht= r.text
    soup = BeautifulSoup(ht) 


    press_statements = soup.find_all('span', {"class": "ModuleHeadline"})
    dates = soup.find_all('span', {"class": "ModuleDate"})


    dates = [datetime.strptime(x.text, "%b %d, %Y").date() for x in dates]
    press_statements = [x.text for x in press_statements]
    pct_changes = [changes_from_press(stock_hist, x, 1) for x in dates]

    data.extend(list(map(list, zip(dates, press_statements, pct_changes))))

data = pd.DataFrame(data, columns=['date', 'press title', '1d change'])

dates_str = [date.strftime("%m/%d/%Y") for date in data['date'].tolist()]

dict_data = {
    'ticker': ticker,
    
    'data': {
        'dates':dates_str,
        'title': data['press title'].tolist(),
        'priceChange': data['1d change'].tolist(),
    },
}

with open(f'./data/{ticker}.json', 'w', encoding='utf-8') as f:
    json.dump(dict_data, f, ensure_ascii=False, indent=4)



C:\Users\Aidan\AppData\Local\Temp\ipykernel_5100\1447083126.py:17: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  while not day_2_price:
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5100\1447083126.py:9: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  while not day_1_price:


In [126]:
data[data['1d change'] == data['1d change'].min()]

,date,press title,1d change
207,2020-03-16,Boeing Human Resources Leader to Depart Company,-20.364104


In [5]:
data

,date,press title,1d change
0,2023-04-05,Boeing to Release First-Quarter Results on Apr...,-0.406870
1,2023-03-30,EVA Air Boosts Sustainable Fleet with Five Add...,1.628768
2,2023-03-30,"Boeing Donates $250,000 to Assist with Tornado...",1.628768
3,2023-03-23,Japan Airlines Selects 737-8 to Grow Sustainab...,-0.187311
4,2023-03-15,Boeing Forecasts Resiliency and Increased Grow...,0.625033
...,...,...,...
577,2011-03-14,Boeing CFO James Bell to Speak at J.P. Morgan ...,-2.123678
578,2011-02-01,Boeing CEO Jim McNerney to Speak at Cowen Aero...,1.408478
579,2011-01-26,Boeing Reports Fourth-Quarter 2010 Results and...,0.807822
580,2011-01-12,Boeing to Release Fourth-Quarter Results on Ja...,0.329382
